In [54]:
import pandas as pd
import numpy as np
import math
import datetime
import time
from ipynb.fs.defs.functions import new_route
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from ipynb.fs.defs.functions import best_index
from ipynb.fs.defs.functions import find_best_position
from ipynb.fs.defs.functions import cities_list
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import check_cities
from ipynb.fs.defs.functions import translate_name
dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/random_mvls/mvls1.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/random_mvls/mvls2.csv", delimiter=";", encoding = "ISO-8859-1")

In [3]:
#Given a list of cities it return False if "to_insert" is already present
#Return True if "to_insert" isn't present and it will give the index of the best position
def check_insert(schedule_day_list, to_insert, dframe):
    if to_insert in schedule_day_list:
        return False, -1
    else:
        #schedule_day_list = cities_list(new_route(schedule_day_list, dframe))
        to_return = best_index(schedule_day_list, new_route(schedule_day_list, dframe), to_insert, dframe)
        return True, to_return

#Given: city list wich to_insert come, the city "to_insert" to be moved, 
#the city list where "to_insert" will be insered, the index of the best position and a dataframe
#the function return the improvement
#If negative the exchange mustn't be done, if positive you gain some time
def improvement(f_l, to_insert, t_l, index, dframe):
    from_list = f_l[:]
    from_time = sum(new_route(from_list, dframe).values)
    from_list.remove(to_insert)
    from_time_removed = sum(new_route(from_list, dframe).values)
    
    to_list = t_l[:]
    to_time = sum(new_route(to_list, dframe).values)
    try:
        to_list.insert(index, to_insert)
    except:
        print(to_list)
    to_time_insert = sum(new_route(to_list, dframe).values)
    
    return (from_time-from_time_removed) + (to_time-to_time_insert)

def to_time_insert(to_insert, t_l, index, dframe):
    to_list = t_l[:]
    try:
        to_list.insert(index, to_insert)
    except:
        print(to_list)
    to_time_insert = sum(new_route(to_list, dframe).values) + 7*(len(to_list)-1)
    return to_time_insert

In [4]:
def get_best_improvement(curr_day, from_vehicle, to_vehicle, best_improvement, best_day, best_vehicle, best_idx, to_insert, curr_f_list, fur1_copy, fur2_copy):
    local_improvement = ""
    if(to_vehicle==1):
        days = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
    else:
        days = ["martedi", "mercoledi", "giovedi"]
        
    for inner_day in days:
        
        if(to_vehicle==1):
            curr_to_list = list(fur1_copy[inner_day].dropna())
        else:
            curr_to_list = list(fur2_copy[inner_day].dropna())
   
        #print(inner_day)
        if inner_day == curr_day and from_vehicle == to_vehicle:
            #print("continua")
            continue
        else:
            boo, index = check_insert(curr_to_list, to_insert, dft)
        if boo:
            local_improvement = improvement(curr_f_list, to_insert, curr_to_list, index, dft)
        else:
            continue
            
        if to_time_insert(to_insert, curr_to_list, index, dft) > 8*60:
            continue
        #If there's an improvement we update "best" variables, in other case we skip to next day
        #print(local_improvement)
        if local_improvement=="":
            continue
        elif local_improvement>best_improvement:
            best_improvement = local_improvement
            best_day = inner_day
            best_vehicle = to_vehicle
            best_idx = index
        else:
            continue
    return best_improvement, best_day, best_vehicle, best_idx

In [57]:
#For every day of every veichle we get every Location
#We compare the improvement removing this location from the day/veichle routing list and inserting it 
#in the other day/veichle routing lists.
#The location will be moved only if the improvement is positive 
#In case of different positive improvement we move in the higher

pudding = 50
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
short_week = ["martedi", "mercoledi", "giovedi"]
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*pudding)
fur1_copy = fur1_copy.join(fur1)

fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([0]*pudding)
fur2_copy = fur2_copy.join(fur2)
week = ["lunedi"]
for day in week:
    #day="lunedi"
    print(day, end="###\n")
    curr_from_list = list(fur1_copy[day].dropna())
    to_insert_list = list(fur1_copy[day].dropna())
    #print(to_insert_list)
    #to_insert_list.remove("Ostiano, Mazzini, 1")
    #curr_from_list.remove("Ostiano, Mazzini, 1")
    cycles = len(to_insert_list)
    #For every location in this schedule
    count = 0
    tot = 0
    startime = time.time()
    for i in range(1, cycles-1):
        
        #print(to_insert_list)
        to_insert = to_insert_list.pop(0)
        #print(to_insert, end=", ")

        best_improvement = -1000
        best_day = ""
        best_vehicle = ""
        best_idx = ""
        #Here we must compare every possible insert in other days
        #Cycle for first veichle
        best_improvement, best_day, best_vehicle, best_idx = get_best_improvement(day, 1, 1, best_improvement, best_day, best_vehicle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)
        best_improvement, best_day, best_vehicle, best_idx = get_best_improvement(day, 1, 2, best_improvement, best_day, best_vehicle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)

        #If best_day isn't modified where isn't an improvement for every day/veichle
        if(best_improvement==-1000):
            continue
        else:
            #Remove from origin list and insert into target list
            curr_from_list.remove(to_insert)
            
            if(best_vehicle==1):
                #print(f"{to_insert} miglioramento veicolo 1 a {best_day}")
                temp_list = list(fur1_copy[best_day].dropna())
                temp_list.insert(best_idx, to_insert)
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)
                fur1_copy[best_day] = pd.Series(temp_list).values

            elif(best_vehicle==2):
                #print(f"{to_insert} miglioramento veicolo 2 a {best_day}")
                temp_list = list(fur2_copy[best_day].dropna())
                temp_list.insert(best_idx, to_insert)
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur2_copy[best_day] =  fur2_copy[best_day].replace(list(fur2_copy[best_day]), temp_list)
                fur2_copy[best_day] = pd.Series(temp_list).values
    padding = pd.Series([np.nan]*(pudding-len(curr_from_list)))
    curr_from_list = pd.concat([pd.Series(curr_from_list), padding], ignore_index=True)
    fur1_copy[day] = pd.Series(curr_from_list).values
    print(round((time.time()-startime)/5, 3))

lunedi###
0.503


In [10]:
results = results[fur1.columns] 
results.to_csv("../Dati/altro/computational_times_OSS.csv", sep=";", encoding = "ISO-8859-1")

,lunedi,martedi,mercoledi,giovedi,venerdi
greedy,"(0.126, 14, 1)","(0.627, 23, 1)","(0.353, 17, 1)","(0.615, 23, 1)","(0.101, 12, 1)"
LS,"(2.374, 16, 10)","(2.02, 15, 5)","(2.3, 16, 7)","(2.332, 16, 3)","(2.034, 14, 4)"
2opt,"(17.511, 16, 2)","(12.013, 15, 3)","(19.358, 16, 3)","(15.941, 16, 8)","(9.73, 14, 4)"
MVLS,"(2.088, 16, 13)","(1.916, 15, 12)","(1.966, 16, 16)","(2.057, 16, 19)","(2.062, 14, 14)"


In [ ]:
#For every day of every veichle we get every Location
#We compare the improvement removing this location from the day/veichle routing list and inserting it 
#in the other day/veichle routing lists.
#The location will be moved only if the improvement is positive 
#In case of different positive improvement we move in the higher
pudding = 50

for day in short_week:
    #day="lunedi"
    print(day, end="###\n")
    curr_from_list = list(fur2_copy[day].dropna())
    to_insert_list = list(fur2_copy[day].dropna())
    #print(to_insert_list)
    #to_insert_list.remove("Ostiano, Mazzini, 1")
    #curr_from_list.remove("Ostiano, Mazzini, 1")
    cycles = len(to_insert_list)
    #For every location in this schedule
    for i in range(1, cycles-1):
        #print(to_insert_list)
        to_insert = to_insert_list.pop(0)
        #print(to_insert, end=", ")
        #to_insert = to_insert_list.pop(0)

        best_improvement = -1000
        best_day = ""
        best_veichle = ""
        best_idx = ""
        #Here we must compare every possible insert in other days
        #Cycle for first veichle
        best_improvement, best_day, best_veichle, best_idx = get_best_improvement(day, 2, 1, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)
        best_improvement, best_day, best_veichle, best_idx = get_best_improvement(day, 2, 2, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list, fur1_copy, fur2_copy)

        #If best_day isn't modified where isn't an improvement for every day/veichle
        if(best_improvement==-1000):
            continue
        else:
            #Remove from origin list and insert into target list
            curr_from_list.remove(to_insert)
            
            if(best_veichle==1):
                #print(f"{to_insert} miglioramento veicolo 1 a {best_day}")
                temp_list = list(fur1_copy[best_day].dropna())
                temp_list.insert(best_idx, to_insert)
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)
                fur1_copy[best_day] = pd.Series(temp_list).values

            elif(best_veichle==2):
                #print(f"{to_insert} miglioramento veicolo 2 a {best_day}")
                temp_list = list(fur2_copy[best_day].dropna())
                temp_list.insert(best_idx, to_insert)
                #temp_list.append("Ostiano, Mazzini, 1")
                padding = pd.Series([np.nan]*(pudding-len(temp_list)))
                temp_list = pd.concat([pd.Series(temp_list), padding], ignore_index=True)
                #fur2_copy[best_day] =  fur2_copy[best_day].replace(list(fur2_copy[best_day]), temp_list)
                fur2_copy[best_day] = pd.Series(temp_list).values
    padding = pd.Series([np.nan]*(pudding-len(curr_from_list)))
    curr_from_list = pd.concat([pd.Series(curr_from_list), padding], ignore_index=True)
    fur2_copy[day] = pd.Series(curr_from_list).values
#fur2_copy  

martedi###
mercoledi###
giovedi###


In [48]:
fur1 = fur1_copy
fur2 = fur2_copy
time = 0
for d in fur1.columns[1:]:
    time = sum(new_route(fur1[d].dropna(), dft).values) + 7*(len(fur1[d])-1)
    if time > 60 * 8:
        print(f"1, {d}, {time}")
#print(time)

time = 0
for d in fur2.columns[1:]:
    time = sum(new_route(fur2[d].dropna(), dft).values) + 7*(len(fur2[d])-1)
    if time > 60 * 8:
        print(f"2, {d}, {time}")
#print(time)

1, lunedi, 641.0
1, mercoledi, 560.0
2, martedi, 497.0
2, mercoledi, 581.0
2, giovedi, 576.0


In [ ]:
fur1_copy = fur1_copy[["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]]
fur2_copy = fur2_copy[["martedi", "mercoledi", "giovedi"]]
fur1_copy.to_csv("../Dati/turni_f/random_inday/random_day_MVLS_f1.csv", sep=";", index=False, encoding = "ISO-8859-1")
fur2_copy.to_csv("../Dati/turni_f/random_inday/random_day_MVLS_f1.csv", sep=";", index=False, encoding = "ISO-8859-1")

In [7]:
fur1_copy

,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Grumello Cremonese ed Uniti, Roma, 2","Spino d'Adda, Circonvallazione, 13","Castelverde, Ubaldo Ferrari, 15","Cremona, Sesto 39","Cremona, Ugolani Dati, 4"
2,"Crema, Civerchi, 9","Monte Cremasco, Roma, 12","Casalbuttano ed Uniti, Jacini, 23","Grontardo, Roma, 16","Gadesco-Pieve Delmona, Giovanni Lonati 9"
3,"Dovera, Oldrini 2","Capergnanica, XI febbraio, 7","Annicco, Giuseppe Garibaldi, 6","Pescarolo ed Uniti, Giuseppe Mazzini, 73","Vescovato, Roma, 44"
4,"Bagnolo Cremasco, Roma, 3","Ripalta Cremasca, Roma, 5","Casalpusterlengo, Pusterla 7","Vescovato, Roma, 44","Pescarolo ed Uniti, Giuseppe Mazzini, 73"
5,"Palazzo Pignano, Marco Polo, 31","Crema, Civerchi, 9","Crema, Civerchi, 9","Ostiano, Mazzini, 1","Ostiano, Mazzini, 1"
6,"Pandino, Castello, 15","Offanengo, Clavelli Martini, 5","Izano, Gerardo da Izano, 4/C",NaN,NaN
7,"Rivolta d'Adda, Stefano Renzi, 5","Soncino, Dante Alighieri, 23","Romanengo, Guaiarini, 14",NaN,NaN
8,"Spino d'Adda, Circonvallazione, 13","Fiesco, Giuseppe Parini, 10","Soncino, Dante Alighieri, 23",NaN,NaN
9,"San Bassano, Mons. Frosi","Trigolo, Roma, 26","Offanengo, Clavelli Martini, 5",NaN,NaN


In [8]:
fur2_copy

,martedi,mercoledi,giovedi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Cremona, Ugolani Dati, 4","Sesto ed Uniti, Roma, 4","Solarolo Rainerio, Giuseppina, 81"
2,"Cremona, Palosca n. 2","Pizzighettone, Giuseppe Garibaldi, 18","San Giovanni in Croce, Feudatari, 25 A"
3,"Bonemerse, Roma, 25","Casalpusterlengo, Pusterla 7",NaN
4,"Persico Dosimo, Albert ,7","Castelleone, Roma, 67",NaN
5,"Ostiano, Mazzini, 1","Crema, Civerchi, 9",NaN
6,"San Giovanni in Croce, Feudatari, 25 A","Madignano, Portici, 4",NaN
7,"San Giovanni in Croce, Feudatari, 25 A","Manerbio, INSURREZIONE NAZIONALE",NaN
8,NaN,"Ostiano, Mazzini, 1",NaN
9,NaN,"Piadena, Platina, 40",NaN


In [6]:
fur1_copy =  translate_name(dft, pd.read_csv("../Dati/orari/route.csv", delimiter=";", encoding = "ISO-8859-1"))
fur2_copy =  translate_name(dft, pd.read_csv("../Dati/orari/void.csv", delimiter=";", encoding = "ISO-8859-1"))
fur1 =  translate_name(dft, pd.read_csv("../Dati/turni_f/original_start/f1_original.csv", delimiter=";", encoding = "ISO-8859-1"))
fur2 =  translate_name(dft, pd.read_csv("../Dati/turni_f/original_start/f2_original.csv", delimiter=";", encoding = "ISO-8859-1"))
_, freq1, freq2 = check_cities(fur1_copy, fur2_copy, fur1, fur2)

In [8]:
l1 = freq1
l2 = freq2
# l1.sort()
# l2.sort()

print(f"l1: {len(l1)}  l2: {len(l2)}")

# for i, e1 in enumerate(l1): 
#     print(l1[i], l2[i])
#     if(list(set(freq1))[i] != list(set(freq2))[i]):
#         print(list(set(freq1))[i], list(set(freq2))[i])

l1: 89  l2: 89


In [13]:
fur1_copy =  pd.read_csv("../Dati/turni_f/optimized_start/oss_f1.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_copy =  pd.read_csv("../Dati/turni_f/optimized_start/oss_f2.csv", delimiter=";", encoding = "ISO-8859-1")
l = []
for day in ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]:
    l += list(fur1_copy[day].dropna()[1:len(fur1_copy[day].dropna())-1])
for day in ["martedi", "mercoledi", "giovedi"]:
    l += list(fur2_copy[day].dropna()[1:len(fur2_copy[day].dropna())-1])

In [14]:
len(l)

87

In [19]:
l = []
fur1_copy =  pd.read_csv("../Dati/orari/route.csv", delimiter=";", encoding = "ISO-8859-1")
fur2_copy =  pd.read_csv("../Dati/orari/void.csv", delimiter=";", encoding = "ISO-8859-1")
for day in ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]:
    l += list(fur1_copy[day].dropna())

In [24]:
fur1_copy

,lunedi,martedi,mercoledi,giovedi,venerdi
0,sesto ed,MANERBIO,Paderno,manerbio,Capergnanica
1,grumello,"Cremona, Felice Cavallotti, 2",San Bassano,"Cremona, sesto",monte cremasco
2,pizzighettone,Casalpusterlengo,Castelleone,pizzighettone,spino d'adda
3,MONTODINE,"Cremona, Ugolani Dati, 4",Spino d'adda,casalpusterlengo,pandino
4,ripalta cremasca,"Crema, Civerchi",Romanengo,"Cremona, Ugolani Dati, 4",casaletto
5,CAPRALBA,Bonemerse,Pandino,"Cremona, Ugolani Dati",bagnolo
6,CASALE CREMASCO,Sospiro,Palazzo Pignano,"pizzighettone, giuseppe","crema, civerchi"
7,CAMISANO,Vailate,Rivolta D'adda,grontardo,vaiano
8,TRIGOLO,Pianengo,Crema,pescarolo,trescore
9,CASTELLEONE,Vescovato,"Cremona, Palosca",izano,rivolta d'adda


In [ ]:
fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([np.nan]*50)
fur2_copy = fur2_copy.join(fur2)
fur2_copy[["martedi", "mercoledi", "giovedi"]]

In [ ]:
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*50)
fur1_copy = fur1_copy.join(fur1)
temp_list = pd.Series(["ciao"])
padding=pd.Series([np.nan]*(50-len(temp_list)))
temp_list = pd.concat([temp_list, padding], ignore_index=True)
fur1_copy[best_day] = fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)